HDBSCAN is a memory-efficient, hierarchical version of DBSCAN for large scale outliers.<br>
why we use?<br>
dbscan_cluster outputs are,
|cluster|cluster count|
|:----|:-----|
|-1|      399|
 |0|    49525|
 |1      | 20|
 |2|       12|
 |3      | 11|
 |4|       10|
 |5|       13|
 |6|       10|
 1) so we see cluster 0 with taxi count 49525. It is hugh count. HDBSCAN auto-tunes and finds multiple clusters better than DBSCAN with varying density. <br>
 2) In the previous dbscan, entire dataset is not used instead 50000 samples were used to generate 7 clusters. <br>


In [1]:
!pip install -Uqq numba
!wget -nc https://raw.githubusercontent.com/rapidsai/rapidsai-csp-utils/main/colab/rapids-colab.sh
!bash rapids-colab.sh stable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed-ucxx-cu12 0.42.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
dask-cuda 25.2.0 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cudf-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
cuml-cu12 25.2.1 requires numba<0.61.0a0,>=0.59.1, but you have numba 0.61.2 which is incompatible.
--2025-06-24 09:01:39--  https://raw.githubusercontent.com/rapidsai/rapidsai-csp-utils/main/colab/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.git

In [2]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import sklearn
print(sklearn.__version__)
import pandas as pd

1.6.1


In [3]:
from google.colab import files

# Open a file picker dialog to upload one or more files
uploaded_files = files.upload()

Saving cleaned_gps_dataset_20250624_0857.csv to cleaned_gps_dataset_20250624_0857.csv


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install plotly
# !pip install nbformat --upgrade
# !pip install umap
# !pip uninstall umap
!pip install umap-learn

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB ? eta 0:00:00
   ---------------------------------------- 0.0/88.8 kB ? eta -:--:--
   ---------------------------------------- 88.8/88.8 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 15.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.9/2.8 MB 24.4 MB/s eta 0:00:01
   ---------------------------------------  2.8/2.8 MB 22.5 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 20.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---- ----------------------------------- 1.4/12.9 MB 29.6 MB/s eta 0:00:01
   ------- -------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Load clean Data

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Load data
df_cleaned = pd.read_csv("cleaned_gps_dataset_20250624_0857.csv")
# Convert timestamp to numeric (e.g., seconds since epoch)

# df_cleaned['ts_num'] = pd.to_datetime(df_cleaned['timestamp']).astype(np.int64) // 10**9
df_cleaned['ts_num'] = (
    pd.to_datetime(df_cleaned['timestamp']) - pd.to_datetime(df_cleaned['timestamp']).min()
).dt.total_seconds()
df_cleaned.head(3)

,taxiid,timestamp,longitude,latitude,ts_num
0,3305,2008-02-02 13:34:25,116.46689,39.99742,221.0
1,3305,2008-02-02 13:39:23,116.46693,39.99744,519.0
2,3305,2008-02-02 13:39:26,116.46693,39.99744,522.0


# Implement HDBSCAN - Hierarchical Density-Based Spatial Clustering, ***unsupervised learning technique***<br>
1. We choose minimum cluster size based on following understanding <br>
1.a Fine-grained local hotspots (taxi stops, blocks)	100–300 <br>
1.b District-level cluster patterns	500–2000 <br>
1.c Very large, dominant movement zones	5000+ <br>
1.d Too small clusters → too many transitions → LSTM performance drops. <br>
Now, we choose min_cluster_size value based on our need to use the outlier output later to ***GMM & LSTN***. Both need <br> less noisy data. Fine-grained will provide too many clusters that may introduce noise. <br>
**So, our goal of using hdbscan is to preserve core shape of dense regions and only remote noise.** <br>
***Now we choose min_cluster_size = 1000***, because GMM needs clean, compact input → HDBSCAN filters noise & LSTM needs stable clusters over time → larger min_cluster_size avoids noisy flips. <br>
***min_cluster_size=10***, means A cluster must contain at least 10 points (i.e., data samples) to be considered a valid cluster.<br>


In [5]:
# Feature selection: latitude, longitude, and time
features = df_cleaned[['latitude', 'longitude', 'ts_num']]

# ✅ Yes — these 3 features are sufficient for spatiotemporal clustering.

In [6]:
# Standardize features to zero mean and unit variance
features_scaled = StandardScaler().fit_transform(features)
# ✅ Perfect — standardization ensures that latitude, longitude, and time contribute equally to the UMAP embedding.

In [7]:
import torch

if torch.cuda.is_available():
    print("✅ GPU is available!")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU is NOT available.")

✅ GPU is available!
GPU Name: Tesla T4


UMAP Overview
UMAP (Uniform Manifold Approximation and Projection) is typically CPU-bound in most common implementations, including the standard one from the umap-learn package.

GPU acceleration is available, but only via the RAPIDS cuML library (which requires special setup and CUDA-compatible environment).

For medium-sized datasets (e.g., <100k samples and <1000 features), a Colab T4 GPU won't help unless using RAPIDS, and CPU will be sufficient.

In [8]:
from cuml.manifold import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.3, n_epochs=100)
embedding = umap.fit_transform(features_scaled)

[2025-06-24 09:07:44.652] [CUML] [info] Building knn graph using nn descent


Once you have embedding (the 2D UMAP output):

Cluster using HDBSCAN:

In [9]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=200, min_samples=10)
df_cleaned['cluster'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


🔢 1. Basic Cluster Distribution

In [10]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster'].value_counts().sort_index()
print(cluster_counts)

cluster
-1      353938
 0         261
 1         350
 2         404
 3         206
         ...  
 780       386
 781       314
 782      1774
 783       498
 784       282
Name: count, Length: 786, dtype: int64


increase the cluster size

In [11]:
import hdbscan
clusterer = hdbscan.HDBSCAN(min_cluster_size=1000, min_samples=5)
df_cleaned['cluster_1'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [13]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster_1'].value_counts().sort_index()
print(cluster_counts)

cluster_1
-1     13602
 0      1274
 1    781547
Name: count, dtype: int64



Try Higher min_cluster_size (e.g., 3000 or 5000)



In [14]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=5)
df_cleaned['cluster_2'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [15]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster_2'].value_counts().sort_index()
print(cluster_counts)

cluster_2
-1     29230
 0      3505
 1    763688
Name: count, dtype: int64


New approach

In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_cleaned = pd.read_csv("cleaned_gps_dataset_20250624_0857.csv")
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

df_cleaned['ts_num'] = (
    df_cleaned['timestamp'] - df_cleaned['timestamp'].min()
).dt.total_seconds()

scaled_features = StandardScaler().fit_transform(df_cleaned[['latitude', 'longitude', 'ts_num']])
df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']] = scaled_features

# Now amplify time
df_cleaned['time_scaled'] *= 2

df_cleaned.head(20)

,taxiid,timestamp,longitude,latitude,ts_num,lat_scaled,lon_scaled,time_scaled
0,3305,2008-02-02 13:34:25,116.46689,39.99742,221.0,0.322839,0.234605,-3.227213
1,3305,2008-02-02 13:39:23,116.46693,39.99744,519.0,0.322942,0.234763,-3.223223
2,3305,2008-02-02 13:39:26,116.46693,39.99744,522.0,0.322942,0.234763,-3.223183
3,3305,2008-02-02 13:44:25,116.46690,39.99743,821.0,0.322891,0.234644,-3.219180
4,3305,2008-02-02 13:44:28,116.46690,39.99743,824.0,0.322891,0.234644,-3.219140
5,3305,2008-02-02 13:49:27,116.46622,39.99356,1123.0,0.303080,0.231942,-3.215137
6,3305,2008-02-02 13:49:30,116.46622,39.99356,1126.0,0.303080,0.231942,-3.215097
7,3305,2008-02-02 13:54:29,116.46619,39.99353,1425.0,0.302927,0.231823,-3.211094
8,3305,2008-02-02 13:54:32,116.46619,39.99353,1428.0,0.302927,0.231823,-3.211054
9,3305,2008-02-02 13:59:30,116.47245,40.00585,1726.0,0.365992,0.256702,-3.207064


In [22]:
import umap
import hdbscan

# Select scaled features
features_for_clustering = df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']]

from cuml.manifold import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.3, n_epochs=100)
embedding = umap.fit_transform(features_for_clustering)

# # Step 1: UMAP
# umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42)
# embedding = umap_model.fit_transform(features_for_clustering)



[2025-06-24 09:51:31.217] [CUML] [info] Building knn graph using nn descent


In [23]:
# Step 2: HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=5)
df_cleaned['cluster'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [25]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster'].value_counts().sort_index()
print(cluster_counts)

cluster
-1     98142
 0      4820
 1      3536
 2      4561
 3      4204
 4     10381
 5    670779
Name: count, dtype: int64


Makes clustering more selective and may reduce the size of the dominant cluster.

In [26]:
# Step 2: HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=15)
df_cleaned['cluster'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [27]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster'].value_counts().sort_index()
print(cluster_counts)

cluster
-1     44034
 0    747516
 1      4873
Name: count, dtype: int64


# Further amplify time_scaled:

If spatial patterns dominate and you want to emphasize time-based transitions.

<!-- Further amplify time_scaled: -->

<!-- If spatial patterns dominate and you want to emphasize time-based transitions. -->


In [29]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_cleaned = pd.read_csv("cleaned_gps_dataset_20250624_0857.csv")
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

df_cleaned['ts_num'] = (
    df_cleaned['timestamp'] - df_cleaned['timestamp'].min()
).dt.total_seconds()

scaled_features = StandardScaler().fit_transform(df_cleaned[['latitude', 'longitude', 'ts_num']])
df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']] = scaled_features

# Now amplify time
df_cleaned['time_scaled'] *= 6 #6 times

df_cleaned.head(20)

,taxiid,timestamp,longitude,latitude,ts_num,lat_scaled,lon_scaled,time_scaled
0,3305,2008-02-02 13:34:25,116.46689,39.99742,221.0,0.322839,0.234605,-9.681638
1,3305,2008-02-02 13:39:23,116.46693,39.99744,519.0,0.322942,0.234763,-9.669670
2,3305,2008-02-02 13:39:26,116.46693,39.99744,522.0,0.322942,0.234763,-9.669549
3,3305,2008-02-02 13:44:25,116.46690,39.99743,821.0,0.322891,0.234644,-9.657540
4,3305,2008-02-02 13:44:28,116.46690,39.99743,824.0,0.322891,0.234644,-9.657420
5,3305,2008-02-02 13:49:27,116.46622,39.99356,1123.0,0.303080,0.231942,-9.645411
6,3305,2008-02-02 13:49:30,116.46622,39.99356,1126.0,0.303080,0.231942,-9.645290
7,3305,2008-02-02 13:54:29,116.46619,39.99353,1425.0,0.302927,0.231823,-9.633281
8,3305,2008-02-02 13:54:32,116.46619,39.99353,1428.0,0.302927,0.231823,-9.633161
9,3305,2008-02-02 13:59:30,116.47245,40.00585,1726.0,0.365992,0.256702,-9.621192


In [30]:
import umap
import hdbscan

# Select scaled features
features_for_clustering = df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']]

from cuml.manifold import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.3, n_epochs=100)
embedding = umap.fit_transform(features_for_clustering)

[2025-06-24 10:22:54.887] [CUML] [info] Building knn graph using nn descent


In [31]:
# Step 2: HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=5)
df_cleaned['cluster'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [32]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster'].value_counts().sort_index()
print(cluster_counts)

cluster
-1     346586
 0       3460
 1       5192
 2       4211
 3       5026
 4       3166
 5       4008
 6       5744
 7       5758
 8       3179
 9      14955
 10     29172
 11      3176
 12      8719
 13      5066
 14      3094
 15      6935
 16      3373
 17      3686
 18      3762
 19      4165
 20      6256
 21      6775
 22      6856
 23      6913
 24      5023
 25      5904
 26     31316
 27      5022
 28     15672
 29      3940
 30      5682
 31      4188
 32      3460
 33      5595
 34      4664
 35     63639
 36      8152
 37     28476
 38      4134
 39      6674
 40     29616
 41      7998
 42      3194
 43      5755
 44      5850
 45     10019
 46     33217
Name: count, dtype: int64


Fine Tuned Outlier Removal Technique

In [77]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_cleaned = pd.read_csv("cleaned_gps_dataset_20250624_0857.csv")
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

df_cleaned['ts_num'] = (
    df_cleaned['timestamp'] - df_cleaned['timestamp'].min()
).dt.total_seconds()

scaled_features = StandardScaler().fit_transform(df_cleaned[['latitude', 'longitude', 'ts_num']])
df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']] = scaled_features

# Now amplify time
df_cleaned['time_scaled'] *= 8/3 #6 times

df_cleaned.head(20)

,taxiid,timestamp,longitude,latitude,ts_num,lat_scaled,lon_scaled,time_scaled
0,3305,2008-02-02 13:34:25,116.46689,39.99742,221.0,0.322839,0.234605,-4.302950
1,3305,2008-02-02 13:39:23,116.46693,39.99744,519.0,0.322942,0.234763,-4.297631
2,3305,2008-02-02 13:39:26,116.46693,39.99744,522.0,0.322942,0.234763,-4.297577
3,3305,2008-02-02 13:44:25,116.46690,39.99743,821.0,0.322891,0.234644,-4.292240
4,3305,2008-02-02 13:44:28,116.46690,39.99743,824.0,0.322891,0.234644,-4.292187
5,3305,2008-02-02 13:49:27,116.46622,39.99356,1123.0,0.303080,0.231942,-4.286849
6,3305,2008-02-02 13:49:30,116.46622,39.99356,1126.0,0.303080,0.231942,-4.286796
7,3305,2008-02-02 13:54:29,116.46619,39.99353,1425.0,0.302927,0.231823,-4.281458
8,3305,2008-02-02 13:54:32,116.46619,39.99353,1428.0,0.302927,0.231823,-4.281405
9,3305,2008-02-02 13:59:30,116.47245,40.00585,1726.0,0.365992,0.256702,-4.276085


In [78]:
import umap
import hdbscan

# Select scaled features
features_for_clustering = df_cleaned[['lat_scaled', 'lon_scaled', 'time_scaled']]

from cuml.manifold import UMAP
umap = UMAP(n_neighbors=15, min_dist=0.1, n_epochs=2000)
embedding = umap.fit_transform(features_for_clustering)

[2025-06-24 11:52:15.337] [CUML] [info] Building knn graph using nn descent


In [80]:
# Step 2: HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=3000, min_samples=5)
df_cleaned['cluster'] = clusterer.fit_predict(embedding)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [81]:
# Count of points per cluster
cluster_counts = df_cleaned['cluster'].value_counts().sort_index()
print(cluster_counts)

cluster
-1     151847
 0       5854
 1       5735
 2       7040
 3      13357
 4       8927
 5       3316
 6       5234
 7      13457
 8       3676
 9       5575
 10      5299
 11     14794
 12      6439
 13      4568
 14     11286
 15      3208
 16    253610
 17      3132
 18      3091
 19      6565
 20      6445
 21      7641
 22     36126
 23      3360
 24      3824
 25     25353
 26    106426
 27      5303
 28     65935
Name: count, dtype: int64
